### This is a shitty pipeline for shitty RBT SNe data reduction and retrieving the fancy lightcurve 

#### 1. Importing packages 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from __reader__ import *

#### 2. Reading the V- and R- data. Transforming the data to fancy and convenient catalogue

In [2]:
path_to_V = r'E:\progs\fuzzy-chainsaw\2024_2025\Photom\V'
path_to_R = r'E:\progs\fuzzy-chainsaw\2024_2025\Photom\dat\dat'

read_data(path_to_V, 'V_RBT')
read_data(path_to_R, 'R_RBT')

data_V = pd.read_csv('V_RBT.csv')
data_R = pd.read_csv('R_RBT.csv')

data_whole = pd.merge(data_V, data_R, on='id', how='inner')

id_col = data_whole.pop('id')
data_whole.insert(0, 'id', id_col)

data_whole.pop('data_y')
data_whole.pop('num_y')

data_whole['date'] = data_whole['data_x']

data_whole.pop('data_x')

date_col = data_whole.pop('date')
data_whole.insert(1, 'date', date_col)
 
data_whole.reset_index(drop=True, inplace=True)
data_whole.to_csv('RV.csv', index=False)

print(data_whole.shape)
print(data_R.shape)
print(data_V.shape)

(240, 9)
(260, 6)
(288, 6)


#### 4. ugriz -> VR_JC transformations for reference stars

In [3]:
# Lupton et al. 2005 (https://www.sdss3.org/dr8/algorithms/sdssUBVRITransform.php) 

df_rs = pd.read_csv('RS.csv')

df_rs['V_JC_PS'] = df_rs['gMeanApMag'] - 0.5784*(df_rs['gMeanApMag'] - df_rs['rMeanApMag']) - 0.0038
df_rs['R_JC_PS'] = df_rs['rMeanApMag'] - 0.2936*(df_rs['rMeanApMag'] - df_rs['iMeanApMag']) - 0.1439

df_rs['V_JC_PS_err'] = np.sqrt(0.0054**2 + df_rs['gMeanApMagErr']**2 + 0.5784**2*(df_rs['gMeanApMagErr']**2 + df_rs['rMeanApMagErr']**2))
df_rs['R_JC_PS_err'] = np.sqrt(0.0072**2 + df_rs['rMeanApMagErr']**2 + 0.2936**2*(df_rs['rMeanApMagErr']**2 + df_rs['iMeanApMagErr']**2))

df_rs.to_csv('RS.csv', index=False)

In [4]:
df_rs = pd.read_csv('RS.csv', usecols=['num', 'V_JC_PS', 'V_JC_PS_err', 'R_JC_PS', 'R_JC_PS_err'])

standarts_dict = {}
for i, row in df_rs.iterrows():
    standarts_dict.update({df_rs['num'][i] : [df_rs['V_JC_PS'][i], df_rs['V_JC_PS_err'][i], df_rs['R_JC_PS'][i], df_rs['R_JC_PS_err'][i]]})
    
standarts_dict.update({1 : [0, 0, 0, 0]})

print(standarts_dict)

{2: [16.93387121124268, 0.0070901628473458, 16.611486906433107, 0.0078815985501842], 3: [18.27946295928955, 0.0142660929209047, 17.846763305664062, 0.0112259503971785], 4: [18.033068951416016, 0.0113135835815499, 17.705330384826663, 0.0087273037438797], 1: [0, 0, 0, 0]}


In [5]:
df_rv = pd.read_csv('RV.csv')

for i, row in df_rv.iterrows():
    if (df_rv['num_x'][i] % 4) in standarts_dict:
        df_rv.at[i, 'V_JC_PS'] = standarts_dict[df_rv['num_x'][i] % 4][0]
        df_rv.at[i, 'V_JC_PS_err'] = standarts_dict[df_rv['num_x'][i] % 4][1]
        df_rv.at[i, 'R_JC_PS'] = standarts_dict[df_rv['num_x'][i] % 4][2]
        df_rv.at[i, 'R_JC_PS_err'] = standarts_dict[df_rv['num_x'][i] % 4][3]
        
    if df_rv['num_x'][i] % 4 == 0:
        df_rv.at[i, 'V_JC_PS'] = standarts_dict[4][0]
        df_rv.at[i, 'V_JC_PS_err'] = standarts_dict[4][1]
        df_rv.at[i, 'R_JC_PS'] = standarts_dict[4][2]
        df_rv.at[i, 'R_JC_PS_err'] = standarts_dict[4][3]
        
df_rv.to_csv('RV.csv', index=False)

#### 3. Trasforming VR_RBT to VR_JC (???) [3 versions, because someone is a dumbass and cannot normally explain all the algo]

In [6]:
a = -23.698
b = -0.102
c = -0.158

d = -23.578
e = -0.144
f = -0.195

X = 1.2

##### 3.1 My Version: (V_l - R_l) is color of star, which originates from catalogue

In [7]:
df_rv = pd.read_csv('RV.csv')

df_rv['R_JC_RBT_1'] = a + b*(df_rv['V_JC_PS'] - df_rv['R_JC_PS']) + c*X + df_rv['R_RBT']
df_rv['V_JC_RBT_1'] = d + e*(df_rv['V_JC_PS'] - df_rv['R_JC_PS']) + f*X + df_rv['V_RBT']

##### 3.2 General Version: solve first equation, assuming Vl, as given, and then get the Vl* (?). Doesn't make sense at all, but ok

In [8]:
df_rv['R_JC_RBT_2'] = a + b*(df_rv['V_JC_PS']) + c*X + df_rv['R_RBT']/(1+b)
df_rv['V_JC_RBT_2'] = d + e*(df_rv['V_JC_PS'] - df_rv['R_JC_RBT_2']) + f*X + df_rv['V_RBT']

##### 3.3 My version 2.0. Let's solve system of equations, Rl - Ri and Vl - Vi, taking both Rl and Vl, as unknown

In [9]:
df_rv['R_JC_RBT_3'] = df_rv['V_RBT'] + a + b*(((df_rv['V_RBT'] - df_rv['R_RBT']) - (a - d) - (c - f)*X)/(b - e)) + f*X
df_rv['V_JC_RBT_3'] = (((df_rv['V_RBT'] - df_rv['R_RBT']) - (a - d) - (c - f)*X)/(b - e)) + df_rv['R_JC_RBT_3']

In [10]:
for i, row in df_rv.iterrows():
    if df_rv['V_JC_PS'][i] == 0:
        df_rv['R_JC_RBT_1'][i] = 0
        df_rv['V_JC_RBT_1'][i] = 0
        df_rv['R_JC_RBT_2'][i] = 0
        df_rv['V_JC_RBT_2'][i] = 0
        df_rv['R_JC_RBT_3'][i] = 0
        df_rv['V_JC_RBT_3'][i] = 0

C:\Users\ALEZHA_ACTIVE\AppData\Local\Temp\ipykernel_14604\1815989419.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rv['R_JC_RBT_1'][i] = 0
C:\Users\ALEZHA_ACTIVE\AppData\Local\Temp\ipykernel_14604\1815989419.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rv['V_JC_RBT_1'][i] = 0
C:\Users\ALEZHA_ACTIVE\AppData\Local\Temp\ipykernel_14604\1815989419.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rv['R_JC_RBT_2'][i]

In [11]:
df_rv.to_csv('RV.csv', index=False)

#### 6. Correct the SNe flux

In [15]:
df_rv = pd.read_csv('RV.csv')

some_id = []
for i in range(int(len(df_rv)/4)):
    some_id.append(4*i + df_rv['num_x'][4*i] % 4)
    
print(some_id)

[1, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 65, 69, 73, 77, 81, 85, 89, 93, 97, 101, 105, 109, 113, 117, 121, 125, 129, 133, 137, 141, 145, 149, 153, 157, 161, 165, 169, 173, 177, 181, 185, 189, 193, 197, 201, 205, 209, 213, 217, 221, 225, 229, 233, 237]


#### 6.1 Absolute magnitude of SNe

#### 7. Plot the lightcurve 